In [1]:
import pandas as pd
import numpy as np
import fastdtw as dtw
from sklearn.cluster import AgglomerativeClustering
from joblib import Parallel, delayed
from tqdm import tqdm
from tqdm.notebook import tqdm
from sklearn.decomposition import PCA
import multiprocessing
import polars as pl
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

In [2]:
#DATOS_DIR = '~/buckets/b1/datasets/'
DATOS_DIR = '../data/'

# Leer datos
df = pd.read_parquet(DATOS_DIR+'FE_7_dataset.parquet') 
df.columns = df.columns.str.replace(' ', '_').str.replace(r'[^A-Za-z0-9_]', '', regex=True)



In [3]:

df = df[df['product_id'] < 20013]
#df = df.drop(columns=['periodo_min_y', 'periodo_max_y', 'periodo_min_x', 'periodo_max_x'])
df = df.reset_index()


In [4]:
df['periodo'] = pd.to_datetime(df['periodo'].astype(str))
df = pl.from_pandas(df)



In [9]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import polars as pl
from tqdm import tqdm
import logging

# Setup logging
logging.basicConfig(level=logging.INFO)

# Assuming 'df' is your existing dataframe
df = df.sort(['product_id', 'customer_id', 'periodo'])

def create_lstm_features(df, customer_id):
    # Filter data for a specific customer
    customer_data = df.filter(pl.col('customer_id') == customer_id)
    
    # Ensure data is sorted by 'periodo'
    customer_data = customer_data.sort('periodo')

    # Prepare the data for LSTM
    series = customer_data.drop(columns=['tn_2', 'periodo']).to_numpy()
    target = customer_data['tn_2'].to_numpy()
    periods = customer_data['periodo'].to_numpy()
    
    n_features = series.shape[1]
    
    X, y, time_index = [], [], []
    for i in range(len(series) - n_features):
        seq_x, seq_y = series[i:i + n_features], target[i + n_features]
        X.append(seq_x)
        y.append(seq_y)
        time_index.append(periods[i + n_features])
    
    X, y = np.array(X), np.array(y)
    X = X.reshape((X.shape[0], X.shape[1], X.shape[2]))
    
    # Define LSTM model
    model = Sequential()
    model.add(LSTM(50, activation='relu', input_shape=(X.shape[1], X.shape[2])))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
    
    # Fit the model
    model.fit(X, y, epochs=10, verbose=1)  # Reduce epochs for quicker debugging
    
    # Generate LSTM features
    lstm_features = model.predict(X, verbose=0)
    
    return lstm_features, time_index

# Apply the function to all customers
customer_ids = df.select(pl.col('customer_id')).unique().to_series().to_list()
product_ids = df.select(pl.col('product_id')).unique().to_series().to_list()
lstm_features_list = []

for proid in tqdm(product_ids, desc="Processing products"):
    df_p = df.filter(pl.col('product_id') == proid)
    for cid in customer_ids:
        customer_data = df_p.filter(pl.col('customer_id') == cid)
        if customer_data.shape[0] >= len(df.columns) - 2:  # Ensure enough data points
            logging.info(f"Processing product_id: {proid}, customer_id: {cid}")
            lstm_features, time_index = create_lstm_features(df_p, cid)
            for idx, feature in enumerate(lstm_features):
                lstm_features_list.append((proid, cid, feature[0], time_index[idx]))

# Convert list to DataFrame
lstm_features_df = pl.DataFrame({
    'product_id': [item[0] for item in lstm_features_list],
    'customer_id': [item[1] for item in lstm_features_list],
    'lstm_feature': [item[2] for item in lstm_features_list],
    'periodo': [item[3] for item in lstm_features_list]
})


Processing products:   0%|          | 0/12 [00:00<?, ?it/s]

Processing products: 100%|██████████| 12/12 [00:01<00:00, 11.49it/s]


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import polars as pl
from tqdm import tqdm

# Assuming 'df' is your existing dataframe
df = df.sort(['product_id', 'customer_id', 'periodo'])

def create_lstm_features(df, customer_id):
    # Filter data for a specific customer
    customer_data = df.filter(pl.col('customer_id') == customer_id)
    
    # Prepare the data for LSTM
    series = customer_data.drop(columns=['tn_2']).to_numpy()
    target = customer_data['tn_2'].to_numpy()
    
    n_features = series.shape[1]
    
    X, y = [], []
    for i in range(len(series) - n_features):
        seq_x, seq_y = series[i:i + n_features], target[i + n_features]
        X.append(seq_x)
        y.append(seq_y)
    
    X, y = np.array(X), np.array(y)
    X = X.reshape((X.shape[0], X.shape[1], X.shape[2]))
    
    # Define LSTM model
    model = Sequential()
    model.add(LSTM(50, activation='relu', input_shape=(n_features, X.shape[2])))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
    
    # Fit the model
    model.fit(X, y, epochs=50, verbose=0)
    
    # Generate LSTM features
    lstm_features = model.predict(X, verbose=0)
    
    return lstm_features

# Apply the function to all customers
customer_ids = df.select(pl.col('customer_id')).unique().to_series().to_list()
product_ids = df.select(pl.col('product_id')).unique().to_series().to_list()
lstm_features_list = []

for proid in tqdm(product_ids, desc="Processing products"):
    df_p = df.filter(pl.col('product_id') == proid)
    for cid in customer_ids:
        if df_p.filter(pl.col('customer_id') == cid).shape[0] >= len(df.columns) - 1:  # Ensure enough data points
            lstm_features = create_lstm_features(df_p, cid)
            for idx, feature in enumerate(lstm_features):
                lstm_features_list.append((proid, cid, feature[0], idx))

# Convert list to DataFrame
lstm_features_df = pl.DataFrame({
    'product_id': [item[0] for item in lstm_features_list],
    'customer_id': [item[1] for item in lstm_features_list],
    'lstm_feature': [item[2] for item in lstm_features_list],
    'time_index': [item[3] for item in lstm_features_list]
})


Processing products:   0%|          | 0/12 [00:00<?, ?it/s]

Processing products: 100%|██████████| 12/12 [00:01<00:00, 11.49it/s]


In [ ]:

df.to_parquet(DATOS_DIR+'/FE_10_dataset-LSTM.parquet', engine='pyarrow')  
